In [1]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
directory="article"
loader = DirectoryLoader(directory)
data = loader.load()
# 初始化加载器
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=128)
# 切割加载的 document
split_docs = text_splitter.split_documents(data)
print(split_docs[10]) # 先拿出一个小片段看看成没成功

The MIME type of 'article/outdata.txt' is 'application/x-dosexec'. This file type is not currently supported in unstructured.


page_content='苗族古歌对研究苗族古代的历史、政治、经济、宗教、哲学、文学、艺术、祭祀、医学、农学、天文、民俗、军事以及民族迁徙等，具有重要的价值。作为苗族最重要的文化遗产，苗族古歌在加强民族凝聚力、构建和谐社会方面有着现实意义。\n\n非遗名称：苗族古歌（簪汪传），所属类型：民间文学，所属省份：贵州省贵阳市清镇市，简介：nan\n\n非遗名称：布洛陀，所属类型：民间文学，所属省份：广西壮族自治区田阳县，简介：申报地区或单位：广西壮族自治区田阳县' metadata={'source': 'article/outdata.txt'}


In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import IPython
import sentence_transformers

EMBEDDING_MODEL = "embedding_module/dataroot/models/shibing624/text2vec-base-chinese"

embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
embeddings.client = sentence_transformers.SentenceTransformer(embeddings.model_name, device='cuda')

/root/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/root/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(split_docs, embeddings)
db.save_local("Faiss_file/") # 指定Faiss的位置

In [4]:
db = FAISS.load_local("Faiss_file/",embeddings=embeddings,allow_dangerous_deserialization=True)

In [7]:
question = "苗族古歌分布在哪些地方？"

similarDocs = db.similarity_search(question, include_metadata=True, k=2)

for x in similarDocs:
    print(x)
# 由输出可见找出的信息与用户提出的问题匹配度还是挺高的

page_content='苗族主要聚居于贵州东南部、广西大苗山、海南岛及贵州、湖南、湖北、四川、云南、广西等省区的交界地带，苗族民歌是苗民在长期生产生活中创造出来的民族民间音乐形式，根据其内容可分为游方歌（情歌）、酒歌、苦歌、反歌、丧歌、劳动歌、时政歌、儿歌、谜语歌等种类，曲调各不相同。各种民歌形式都是既有传统的歌词，也有即兴编唱的内容。其中苗族飞歌曲调高亢，豪迈奔放，采用独唱、对唱、合唱等形式演唱，深受苗民喜爱。' metadata={'source': 'article/outdata.txt'}
page_content='非遗名称：苗族古歌，所属类型：民间文学，所属省份：湖南省花垣县，简介：申报地区或单位：湖南省花垣县\n\n苗族古歌是苗族古老的口头文学形式，其内容包括开天辟地、战争迁徙、风俗习惯、生产劳动、神话传说、爱情故事、情歌礼辞、丧葬火把、苗医苗药以及天文哲学等。苗族古歌的传播有两种形式：口头演述和抄本流传。' metadata={'source': 'article/outdata.txt'}


In [8]:
from langchain.llms.base import LLM
from typing import Any, List, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch

class baichuan2_LLM(LLM):
    # 基于本地 Baichuan 自定义 LLM 类
    tokenizer : AutoTokenizer = None
    model: AutoModelForCausalLM = None

    def __init__(self, model_path :str):
        # model_path: Baichuan-13B-chat模型路径
        # 从本地初始化模型
        super().__init__()
        print("正在从本地加载模型...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True,offload_folder = "./Baichuan2/offload")
        self.model.generation_config = GenerationConfig.from_pretrained(model_path)
        self.model = self.model.eval()
        print("完成本地模型的加载")

    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
         # 重写调用函数
        messages = [
            {"role": "user", "content": prompt}
        ]
         # 重写调用函数
        response= self.model.chat(self.tokenizer, messages)
        return response
        
    @property
    def _llm_type(self) -> str:
        return "baichuan2_LLM"

In [9]:
llm = baichuan2_LLM("./model/Baichuan2-7B-Chat/models")

正在从本地加载模型...
完成本地模型的加载


In [10]:
# 直接调用LangChain的RetrievalQA，实现基于上下文的问答，省去写prompt的步骤
from langchain.chains import RetrievalQA
import IPython

retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)


In [11]:
query = "苗族古歌分布在哪些省份？"
print(qa.run(query))

/root/miniconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
/root/miniconda3/lib/python3.10/site-packages/torch/backends/cuda/__init__.py:342: FutureWarning: torch.backends.cuda.sdp_kernel() is deprecated. In the future, this context manager will be removed. Please see, torch.nn.attention.sdpa_kernel() for the new context manager, wi

苗族古歌分布的省份有湖南省花垣县、吉首市、凤凰县、保靖县、古丈县、泸溪县以及与之接壤的贵州、重庆和湖北等地。


In [12]:
retriever = db .as_retriever()
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever
)
response = qa.run('苗族古歌是什么？')
print(response)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
/root/miniconda3/lib/python3.10/site-packages/torch/backends/cuda/__init__.py:342: FutureWarning: torch.backends.cuda.sdp_kernel() is deprecated. In the future, this context manager will be removed. Please see, torch.nn.attention.sdpa_kernel() for the new context manager, with updated signature.
  warnings.warn(


苗族古歌是苗族古老的口头文学形式，其内容包括开天辟地、战争迁徙、风俗习惯、生产劳动、神话传说、爱情故事、情歌礼辞、丧葬火把、苗医苗药以及天文哲学等。苗族古歌的传播有两种形式：口头演述和抄本流传。
